#Create the environment

In [6]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [7]:
%cd /content/drive/My Drive/ESoWC

/content/drive/My Drive/ESoWC


In [8]:
import pandas as pd
import xarray as xr

import numpy as np
import pandas as pd
from sklearn import preprocessing
import seaborn as sns

In [9]:
fn_land = 'Data/land_cover_data.nc'
fn_weather = 'Data/05_2019_weather_and_CO_for_model.nc'
fn_conc = 'Data/totalcolConcentretations_featured.nc'
fn_traffic = 'Data/emissions_traffic_hourly_merged.nc'

#Load datasets

##Land

In [10]:
# Open netCDF file 
land = xr.open_dataset(fn_land)
land

<xarray.Dataset>
Dimensions:       (lat: 33, lon: 33, time: 30)
Coordinates:
  * time          (time) datetime64[ns] 2019-05-01 2019-05-02 ... 2019-05-30
  * lon           (lon) float64 4.0 4.25 4.5 4.75 5.0 ... 11.25 11.5 11.75 12.0
  * lat           (lat) float64 43.0 43.25 43.5 43.75 ... 50.25 50.5 50.75 51.0
Data variables:
    height        (time, lat, lon) float32 ...
    built         (time, lat, lon) float32 ...
    NO emissions  (time, lat, lon) float32 ...

In [11]:
land_fixed = land.drop_vars('NO emissions') #They are already in the weather dataset
hours = np.arange(0,24,1)
land_fixed = land_fixed.expand_dims({'Hours':hours})
land_fixed = land_fixed.assign_coords(time=land_fixed.time.dt.day)
land_fixed = land_fixed.rename({'time':'Days'})
land_fixed = land_fixed.rename({'lon':'longitude'})
land_fixed = land_fixed.rename({'lat':'latitude'})
land_fixed = land_fixed.transpose('latitude','longitude','Days','Hours')  
land_fixed

<xarray.Dataset>
Dimensions:    (Days: 30, Hours: 24, latitude: 33, longitude: 33)
Coordinates:
  * Hours      (Hours) int64 0 1 2 3 4 5 6 7 8 9 ... 15 16 17 18 19 20 21 22 23
  * Days       (Days) int64 1 2 3 4 5 6 7 8 9 10 ... 22 23 24 25 26 27 28 29 30
  * longitude  (longitude) float64 4.0 4.25 4.5 4.75 ... 11.25 11.5 11.75 12.0
  * latitude   (latitude) float64 43.0 43.25 43.5 43.75 ... 50.5 50.75 51.0
Data variables:
    height     (latitude, longitude, Days, Hours) float32 0.0 0.0 ... nan nan
    built      (latitude, longitude, Days, Hours) float32 0.0 0.0 ... nan nan

##Weather

In [12]:
weather = xr.open_dataset(fn_weather)
weather

<xarray.Dataset>
Dimensions:            (Days: 31, Hours: 24, latitude: 33, longitude: 33)
Coordinates:
  * latitude           (latitude) float64 43.0 43.25 43.5 ... 50.5 50.75 51.0
  * longitude          (longitude) float64 4.0 4.25 4.5 4.75 ... 11.5 11.75 12.0
  * Days               (Days) int64 1 2 3 4 5 6 7 8 ... 24 25 26 27 28 29 30 31
  * Hours              (Hours) int64 0 1 2 3 4 5 6 7 ... 16 17 18 19 20 21 22 23
Data variables: (12/18)
    EMISSIONS_2019     (latitude, longitude, Days, Hours) float64 ...
    u10                (latitude, longitude, Days, Hours) float64 ...
    v10                (latitude, longitude, Days, Hours) float64 ...
    hcc                (latitude, longitude, Days, Hours) float64 ...
    lcc                (latitude, longitude, Days, Hours) float64 ...
    tcc                (latitude, longitude, Days, Hours) float64 ...
    ...                 ...
    tcwv               (latitude, longitude, Days, Hours) float64 ...
    tcw                (latitude, longitude, Days, Hours) float64 ...
    tot_wind           (latitude, longitude, Days, Hours) float64 ...
    tmp_shift_8        (latitude, longitude, Days, Hours) float64 ...
    tot_wind_shift_12  (latitude, longitude, Days, Hours) float64 ...
    rel_hum_shift_8    (latitude, longitude, Days, Hours) float64 ...

In [13]:
#This variable is too much correlated with the tcw
weather_fixed = weather.drop_vars('tcwv')

weather_fixed = weather_fixed.transpose('latitude','longitude','Days','Hours')  
weather_fixed

<xarray.Dataset>
Dimensions:            (Days: 31, Hours: 24, latitude: 33, longitude: 33)
Coordinates:
  * latitude           (latitude) float64 43.0 43.25 43.5 ... 50.5 50.75 51.0
  * longitude          (longitude) float64 4.0 4.25 4.5 4.75 ... 11.5 11.75 12.0
  * Days               (Days) int64 1 2 3 4 5 6 7 8 ... 24 25 26 27 28 29 30 31
  * Hours              (Hours) int64 0 1 2 3 4 5 6 7 ... 16 17 18 19 20 21 22 23
Data variables: (12/17)
    EMISSIONS_2019     (latitude, longitude, Days, Hours) float64 ...
    u10                (latitude, longitude, Days, Hours) float64 ...
    v10                (latitude, longitude, Days, Hours) float64 ...
    hcc                (latitude, longitude, Days, Hours) float64 ...
    lcc                (latitude, longitude, Days, Hours) float64 ...
    tcc                (latitude, longitude, Days, Hours) float64 ...
    ...                 ...
    rel_hum            (latitude, longitude, Days, Hours) float64 ...
    tcw                (latitude, longitude, Days, Hours) float64 ...
    tot_wind           (latitude, longitude, Days, Hours) float64 ...
    tmp_shift_8        (latitude, longitude, Days, Hours) float64 ...
    tot_wind_shift_12  (latitude, longitude, Days, Hours) float64 ...
    rel_hum_shift_8    (latitude, longitude, Days, Hours) float64 ...

##Conc

In [14]:
conc = xr.open_dataset(fn_conc)
conc

<xarray.Dataset>
Dimensions:           (latitude: 33, longitude: 33, time: 708)
Coordinates:
  * latitude          (latitude) float64 43.0 43.25 43.5 ... 50.5 50.75 51.0
  * longitude         (longitude) float64 4.0 4.25 4.5 4.75 ... 11.5 11.75 12.0
  * time              (time) datetime64[ns] 2019-05-01T01:00:00 ... 2019-05-3...
Data variables:
    NO_tc             (latitude, longitude, time) float64 ...
    CO2_tc            (latitude, longitude, time) float64 ...
    CH4_tc            (latitude, longitude, time) float64 ...
    NO2_tc            (latitude, longitude, time) float64 ...
    CO_tc             (latitude, longitude, time) float64 ...
    O3_tc             (latitude, longitude, time) float64 ...
    NO_tc_add_trend   (latitude, longitude, time) float64 ...
    CO2_tc_add_trend  (latitude, longitude, time) float64 ...
    CH4_tc_add_trend  (latitude, longitude, time) float64 ...
    NO2_tc_add_trend  (latitude, longitude, time) float64 ...
    CO_tc_add_trend   (latitude, longitude, time) float64 ...
    O3_tc_add_trend   (latitude, longitude, time) float64 ...

In [15]:
dataframe = conc.to_dataframe()

dataframe['Days'] = dataframe.index.get_level_values("time").day 
dataframe['Hours'] = dataframe.index.get_level_values("time").hour
dataframe.set_index('Days', append=True, inplace=True)
dataframe.set_index('Hours', append=True, inplace  =True)
dataframe = dataframe.reset_index()
dataframe= dataframe.drop(["time"], axis=1)
dataframe = dataframe.set_index(['latitude', 'longitude', 'Days', 'Hours'])

dataframe.head()

NO_tc  ...  O3_tc_add_trend
latitude longitude Days Hours                ...                 
43.0     4.0       1    1               NaN  ...              NaN
                        2      4.242915e-07  ...         0.007345
                        3      6.189903e-07  ...         0.007345
                        4      8.136891e-07  ...         0.007346
                        5      1.008388e-06  ...         0.007346

[5 rows x 12 columns]

In [16]:
conc_fidex=dataframe.to_xarray()
conc_fidex

<xarray.Dataset>
Dimensions:           (Days: 30, Hours: 24, latitude: 33, longitude: 33)
Coordinates:
  * latitude          (latitude) float64 43.0 43.25 43.5 ... 50.5 50.75 51.0
  * longitude         (longitude) float64 4.0 4.25 4.5 4.75 ... 11.5 11.75 12.0
  * Days              (Days) int64 1 2 3 4 5 6 7 8 9 ... 23 24 25 26 27 28 29 30
  * Hours             (Hours) int64 0 1 2 3 4 5 6 7 ... 16 17 18 19 20 21 22 23
Data variables:
    NO_tc             (latitude, longitude, Days, Hours) float64 nan nan ... nan
    CO2_tc            (latitude, longitude, Days, Hours) float64 nan nan ... nan
    CH4_tc            (latitude, longitude, Days, Hours) float64 nan nan ... nan
    NO2_tc            (latitude, longitude, Days, Hours) float64 nan nan ... nan
    CO_tc             (latitude, longitude, Days, Hours) float64 nan nan ... nan
    O3_tc             (latitude, longitude, Days, Hours) float64 nan nan ... nan
    NO_tc_add_trend   (latitude, longitude, Days, Hours) float64 nan nan ... nan
    CO2_tc_add_trend  (latitude, longitude, Days, Hours) float64 nan nan ... nan
    CH4_tc_add_trend  (latitude, longitude, Days, Hours) float64 nan nan ... nan
    NO2_tc_add_trend  (latitude, longitude, Days, Hours) float64 nan nan ... nan
    CO_tc_add_trend   (latitude, longitude, Days, Hours) float64 nan nan ... nan
    O3_tc_add_trend   (latitude, longitude, Days, Hours) float64 nan nan ... nan

#Traffic

In [17]:
traffic = xr.open_dataset(fn_traffic)
traffic

<xarray.Dataset>
Dimensions:    (hour: 24, latitude: 60, longitude: 61)
Coordinates:
  * longitude  (longitude) float32 4.0 4.1 4.2 4.3 4.4 ... 9.6 9.7 9.8 9.9 10.0
  * latitude   (latitude) float32 43.05 43.15 43.25 43.35 ... 48.75 48.85 48.95
  * hour       (hour) int64 0 1 2 3 4 5 6 7 8 9 ... 15 16 17 18 19 20 21 22 23
Data variables:
    emissions  (hour, latitude, longitude) float32 ...
    traffic    (hour, latitude, longitude) float32 ...

In [18]:
traffic_fixed=traffic.drop_vars('emissions')
lat_bins = np.arange(43,51.25,0.25)
lon_bins = np.arange(4,12.25,0.25)
traffic_fixed = traffic_fixed.sortby(['latitude','longitude','hour'])  
traffic_fixed = traffic_fixed.interp(latitude=lat_bins, longitude=lon_bins, method="linear")
days = np.arange(1,32,1)
traffic_fixed=traffic_fixed.expand_dims({'Days':days})
traffic_fixed=traffic_fixed.rename({'hour':'Hours'})
traffic_fixed=traffic_fixed.transpose('latitude','longitude','Days','Hours')  
traffic_fixed

<xarray.Dataset>
Dimensions:    (Days: 31, Hours: 24, latitude: 33, longitude: 33)
Coordinates:
  * Days       (Days) int64 1 2 3 4 5 6 7 8 9 10 ... 23 24 25 26 27 28 29 30 31
  * Hours      (Hours) int64 0 1 2 3 4 5 6 7 8 9 ... 15 16 17 18 19 20 21 22 23
  * latitude   (latitude) float64 43.0 43.25 43.5 43.75 ... 50.5 50.75 51.0
  * longitude  (longitude) float64 4.0 4.25 4.5 4.75 ... 11.25 11.5 11.75 12.0
Data variables:
    traffic    (latitude, longitude, Days, Hours) float64 nan nan ... nan nan

#Merge

In [19]:
tot_dataset = weather_fixed.merge(land_fixed)
tot_dataset = tot_dataset.merge(conc_fidex)
tot_dataset = tot_dataset.merge(traffic_fixed)

tot_dataset

<xarray.Dataset>
Dimensions:            (Days: 31, Hours: 24, latitude: 33, longitude: 33)
Coordinates:
  * Days               (Days) int64 1 2 3 4 5 6 7 8 ... 24 25 26 27 28 29 30 31
  * latitude           (latitude) float64 43.0 43.25 43.5 ... 50.5 50.75 51.0
  * longitude          (longitude) float64 4.0 4.25 4.5 4.75 ... 11.5 11.75 12.0
  * Hours              (Hours) int64 0 1 2 3 4 5 6 7 ... 16 17 18 19 20 21 22 23
Data variables: (12/32)
    EMISSIONS_2019     (latitude, longitude, Days, Hours) float64 ...
    u10                (latitude, longitude, Days, Hours) float64 ...
    v10                (latitude, longitude, Days, Hours) float64 ...
    hcc                (latitude, longitude, Days, Hours) float64 ...
    lcc                (latitude, longitude, Days, Hours) float64 ...
    tcc                (latitude, longitude, Days, Hours) float64 ...
    ...                 ...
    CO2_tc_add_trend   (latitude, longitude, Days, Hours) float64 nan ... nan
    CH4_tc_add_trend   (latitude, longitude, Days, Hours) float64 nan ... nan
    NO2_tc_add_trend   (latitude, longitude, Days, Hours) float64 nan ... nan
    CO_tc_add_trend    (latitude, longitude, Days, Hours) float64 nan ... nan
    O3_tc_add_trend    (latitude, longitude, Days, Hours) float64 nan ... nan
    traffic            (latitude, longitude, Days, Hours) float64 nan ... nan

In [20]:
tot_dataset.to_netcdf('Data/dataset_complete_for_model_CO.nc', 'w', 'NETCDF4')

#Check

In [21]:
weather_fixed.to_dataframe().isnull().sum()

EMISSIONS_2019       26136
u10                   2178
v10                   2178
hcc                   2178
lcc                   2178
tcc                   2178
cvl                   2178
cvh                   2178
sp                    2178
tmp                   2178
sp_hum                2178
rel_hum               2178
tcw                   2178
tot_wind              2178
tmp_shift_8           2186
tot_wind_shift_12     2190
rel_hum_shift_8       2186
dtype: int64

In [22]:
land_fixed.to_dataframe().isnull().sum()

height    23760
built     23760
dtype: int64

In [23]:
conc_fidex.to_dataframe().isnull().sum()

NO_tc               13073
CO2_tc              13073
CH4_tc              13073
NO2_tc              13073
CO_tc               13073
O3_tc               13073
NO_tc_add_trend     13073
CO2_tc_add_trend    13073
CH4_tc_add_trend    13073
NO2_tc_add_trend    13073
CO_tc_add_trend     13073
O3_tc_add_trend     13073
dtype: int64

In [24]:
traffic_fixed.to_dataframe().isnull().sum()

traffic    382416
dtype: int64